In [39]:
import glob
import os
from pathlib import Path
import os, sqlite3, glob
from subprocess import call

def pre_process(candidate, pre_process_type):
    log_filename = '{}/log.txt'.format(candidate)
    logfile = open(log_filename, 'wb')
    
    ######## Diffusion files
    # Dicom to nifti conversion
    if pre_process_type == 'compute_nifti_DTI':
        if len(glob.glob('{}/diffusion.nii.gz'.format(candidate))) == 0:
            ls = glob.glob('{}/*{}*'.format(candidate, DWI_KEY))
            print('Converting diffusion DICOM to Nifti...')
            if len(ls) != 1:
                logfile.write(b'DWI file is not present!')
                return 2
            dwi_dicom_filename = ls[0].split('/')[-1]

            #if call([dcm2nii, '-a', 'Y', '-b', '{}'.format(DCM2NII_CFG), '{}/{}.dcm'.format(candidate, dwi_dicom_filename)],
            #        stdout=logfile, stderr=logfile):
            if call([dcm2nii, '-f', 'diffusion', '-ba', 'y', '-z', 'i', '-6', '-s', 'y',
                     '{}/{}'.format(candidate, dwi_dicom_filename)], stdout=logfile, stderr=logfile):
                logfile.write(b'Fail while trying DWI DICOM to NifTi!')
                return 2

    # Brain extraction diffusion
    elif pre_process_type == 'brain_extraction_DTI':
        if len(glob.glob('{}/nodif_brain_mask.nii.gz'.format(candidate)))==0:
            print('Extracting diffusion brain...')
            if call(['bet', '{}/diffusion'.format(candidate), '{}/nodif_brain'.format(candidate), '-m', '-f',
                     '0.1', '-R'], stdout=logfile, stderr=logfile):
                logfile.write(b'Fail while extracting brain!')
                return 2
            call(['rm', '{}/nodif_brain.nii.gz'.format(candidate)], stdout=logfile, stderr=logfile)

    # DTI and scalar maps calculation
    elif pre_process_type == 'compute_DTI':
        if len(glob.glob('{}/dti_L1.nii.gz'.format(candidate)))==0:
            print('Calculating DTI...')
            if call(['dtifit', '-k', '{}/diffusion'.format(candidate), '-o', '{}/dti'.format(candidate),'-m',
                     '{}/nodif_brain_mask'.format(candidate), '-r', '{}/diffusion.bvec'.format(candidate),
                     '-b', '{}/diffusion.bval'.format(candidate)], stdout=logfile, stderr=logfile):
                logfile.write(b'Fail while generating DTI volume!')
                return 2
            
    # Dicom to nifti conversion
    elif pre_process_type == 'compute_nifti_T1':
        if len(glob.glob('{}/T1.nii.gz'.format(candidate))) == 0:
            ls = glob.glob('{}/*{}*'.format(candidate, T1_KEY))
            print('Converting T1 DICOM to Nifti...')
            if len(ls) != 1:
                logfile.write(b'T1 file is not present!')
                return 2
            t1_dicom_filename = ls[0].split('/')[-1]

            if call([dcm2nii, '-f', 'T1', '-ba', 'y', '-z', 'i', '-6', '-s', 'y',
                     '{}/{}'.format(candidate, t1_dicom_filename)], stdout=logfile, stderr=logfile):
                logfile.write(b'Fail while trying T1 DICOM to NifTi.')
                return 2
    
    else:
        print('Unknown pre-processing')
        return 1
    return 0

### Get folder directions

Files downloaded from OASIS3 dataset
Configurations:
* 1) Scan Formats: BIDS and NIFTI
* 2) Scan Types: dwi
* 3) Download options: Include subject in file paths / Simplify downloaded archive structure

In [60]:
PARENT_FOLDER = '/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140347'

files = [i for i in glob.glob('{}/**/*.nii.gz'.format(PARENT_FOLDER), recursive=True)]
files

['/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140347/OAS30001/OAS30001_MR_d0757/dwi1/NIFTI/sub-OAS30001_ses-d0757_dwi.nii.gz',
 '/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140347/OAS30003/OAS30003_MR_d1631/dwi1/NIFTI/sub-OAS30003_ses-d1631_run-01_dwi.nii.gz',
 '/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140347/OAS30003/OAS30003_MR_d1631/dwi2/NIFTI/sub-OAS30003_ses-d1631_run-02_dwi.nii.gz',
 '/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140347/OAS30004/OAS30004_MR_d2232/dwi1/NIFTI/sub-OAS30004_ses-d2232_run-01_dwi.nii.gz',
 '/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140347/OAS30004/OAS30004_MR_d2232/dwi2/NIFTI/sub-OAS30004_ses-d2232_run-02_dwi.nii.gz',
 '/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140347/OAS30005/OAS30005_MR_d2384/dwi1/NIFTI/sub-OAS30005_ses-d2384_run-01_dwi.nii.gz',
 '/media/thais-caldeira/HD/Unicamp/Mestrado/thais.caldeira-20210107_140

### Backup and rename files

In [84]:
MAIN_FOLDER = '/media/thais-caldeira/HD/Unicamp/Mestrado/OASIS3'
TO_PROCESS = []

for file in files:
    
    print('Backup file: {}'.format(os.path.basename(file)))
    
    # Get folder name
    NEW_FOLDER = os.path.basename(file).rsplit('.')[0]
    
    # Create folder inside MAIN_FOLDER
    NEW_FOLDER_PATH = '{}/{}'.format(MAIN_FOLDER, NEW_FOLDER)
    Path(NEW_FOLDER_PATH).mkdir(parents=True, exist_ok=True)
    TO_PROCESS.append(NEW_FOLDER_PATH)
    
    # Copy dwi file into new folder
    ORIGIN = file
    DESTIN = '{}/{}/{}'.format(MAIN_FOLDER, NEW_FOLDER, 'diffusion.nii.gz')

    os.system('cp {} {}'.format(ORIGIN, DESTIN))
    
    # Copy bval, bvec, json
    for ext in ['bval', 'bvec', 'json']:
        ORIGIN = '{}/BIDS/{}.{}'.format(os.path.dirname(os.path.dirname(file)), os.path.basename(file).rsplit('.')[0], ext)
        DESTIN = '{}/{}/diffusion.{}'.format(MAIN_FOLDER, NEW_FOLDER, ext)

        os.system('cp {} {}'.format(ORIGIN, DESTIN))

Backup file: sub-OAS30001_ses-d0757_dwi.nii.gz
Backup file: sub-OAS30003_ses-d1631_run-01_dwi.nii.gz
Backup file: sub-OAS30003_ses-d1631_run-02_dwi.nii.gz
Backup file: sub-OAS30004_ses-d2232_run-01_dwi.nii.gz
Backup file: sub-OAS30004_ses-d2232_run-02_dwi.nii.gz
Backup file: sub-OAS30005_ses-d2384_run-01_dwi.nii.gz
Backup file: sub-OAS30005_ses-d2384_run-02_dwi.nii.gz
Backup file: sub-OAS30006_ses-d0373_dwi.nii.gz


### Preprocess: Extract brain mask, and DWI to DTI

In [86]:
#pre_process_list = ['compute_nifti_DTI', 'brain_extraction_DTI', 'compute_DTI']
pre_process_list = ['brain_extraction_DTI', 'compute_DTI']

for sub_folder in TO_PROCESS:

    print('Pre-processing --> ', sub_folder)
    subject_error = []
    for pre_process_type in pre_process_list:
        pp = pre_process(sub_folder + '/', pre_process_type)
        if pp == 2:
            subject_error.append(sub_folder + '_' + pre_process_type)
        else:
            pass
            
    print('----------------------------------')
print('Finished.')

Pre-processing -->  /media/thais-caldeira/HD/Unicamp/Mestrado/OASIS3/sub-OAS30001_ses-d0757_dwi
Extracting diffusion brain...
Calculating DTI...
----------------------------------
Pre-processing -->  /media/thais-caldeira/HD/Unicamp/Mestrado/OASIS3/sub-OAS30003_ses-d1631_run-01_dwi
Extracting diffusion brain...
Calculating DTI...
----------------------------------
Pre-processing -->  /media/thais-caldeira/HD/Unicamp/Mestrado/OASIS3/sub-OAS30003_ses-d1631_run-02_dwi
Extracting diffusion brain...
Calculating DTI...
----------------------------------
Pre-processing -->  /media/thais-caldeira/HD/Unicamp/Mestrado/OASIS3/sub-OAS30004_ses-d2232_run-01_dwi
Extracting diffusion brain...
Calculating DTI...
----------------------------------
Pre-processing -->  /media/thais-caldeira/HD/Unicamp/Mestrado/OASIS3/sub-OAS30004_ses-d2232_run-02_dwi
Extracting diffusion brain...
Calculating DTI...
----------------------------------
Pre-processing -->  /media/thais-caldeira/HD/Unicamp/Mestrado/OASIS3/sub